This is a script for geocoding and plotting spatial information from an EXCEL spreadsheet with an "Addresses" column. The API used is **GeoNames.**

GeoNames is mainly using REST APIs. It offers 40 different webservices.

**Geocoder** for Python supports the following ones:

*   (geocoding) retrieve GeoNames’s geocoded data from a query string, and 
various filters
*   (details) retrieve all geonames data for a given geonames_id
*   (children) retrieve the hierarchy of a given geonames_id
*   (hierarchy) retrieve all children for a given geonames_id

Full documentation: https://geocoder.readthedocs.io/providers/GeoNames.html

The first step is to get COLAB working:


In [ ]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


A file path needs to be defined for storing input or output files linked with this script:

In [ ]:
directory="/content/drive/My Drive/Colab_FASoS/" ## add your own folder name

Now we can install packages that are not part of Python's standard distribution but are necessary for geocoding and plotting maps. There will most likely be a dependency error for NumPy, but the script should still work.

In [ ]:
## install packages that are not part of Python's standard distribution

!pip install geocoder
!pip install basemap
!pip install ipyleaflet
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.6/860.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 78.8 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now that all packages are installed, we can read the input data (in this case from Github or Google Drive) and display the content in a table.

In [ ]:
## import relevant packages for geocoding as well as reading and writing data
import pandas as pd
import numpy as np # e.g. for working with empty cells
import geocoder
# command needed for correct plotting in Jupyter Notebooks:
%matplotlib inline 
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import os
import json
from geojson import Feature, FeatureCollection, Point

## geocode data from spreadsheet

## input addresses in EXCEL format and read
## OPTION 1: from Github as raw file
##infile="https://github.com/MonikaBarget/GeoHumTutorials/blob/master/Colab_Geocoding/Addresses_AP3.xlsx?raw=true"

## OPTION 2: from your Google Drive as EXCEL FILE
infile=directory+"Mainz2.xlsx"

## OPTION 3: from your Google Drive as CSV FILE
#infile=directory+"ATLASSES & COSMOGRAPHIES_utf-8.csv"

## read if EXCEL
addresses_df = pd.read_excel(infile)
display(addresses_df)

## read if CSV
#addresses_df = pd.read_csv(infile, encoding="utf-8", delimiter=";") # delimiter=None # encoding=None # encoding_errors='strict'
#display(addresses_df)

,Unnamed: 0.1,Unnamed: 0,place_old,address,comment_fs,suffix,community,region_1,region_2,continent,variant_1,variant_2,variant_3,Source,addresses_full,latitudes,longitudes,ids,geonames address
0,0,0,Bamberg,Bamberg,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,ProfAPI,"Bamberg, Europe",49.89873,10.90067,2952984.0,Bamberg
1,1,1,?eská Lípa,Böhmisch-Leipa,NaN,NaN,NaN,NaN,NaN,Europe,Böhmisch-Leipa,NaN,NaN,G / 1694 Dezember 7,"Böhmisch-Leipa, Europe",50.68551,14.53764,3077929.0,Česká Lípa
2,2,2,?eský Dub,Böhmisch-Aicha,NaN,NaN,NaN,NaN,NaN,Europe,Böhmisch-Aicha,NaN,NaN,G / 1696 Februar 14,"Böhmisch-Aicha, Europe",50.66054,14.99617,3077895.0,Český Dub
3,3,3,berlingen,Überlingen,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Überlingen, Europe",47.76977,9.17136,2820577.0,Überlingen
4,4,4,A?manstadianus,Assamstadt,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Assamstadt, Europe",49.42806,9.68611,2955050.0,Assamstadt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,3747,3747,Tilendropiensis,Boles?awiec,unklar,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Boles?awiec, Europe",NaN,NaN,NaN,NaN
3750,3748,3748,Triltzenbergensis,Triltzenberg,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Triltzenberg, Europe",NaN,NaN,NaN,NaN
3751,3751,3751,Wengeskirchen,Wengeskirchen,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Wengeskirchen, Europe",NaN,NaN,NaN,NaN
3752,3752,3752,Widenfeldensis,"Weidenfeld, Bad Orb",fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Weidenfeld, Bad Orb, Europe",NaN,NaN,NaN,NaN


Now we will use the Pandas package to read the content of the address column to a so-called DataFrame. A DataFrame is a data structure that organizes data into a 2-dimensional table of rows and columns, much like a spreadsheet. This 2-dimensional structure is often used to manipulate data with programming languages. Our "manipulation" is the act of geocoding.

In [ ]:
# read information from address column to dataframe
addresses_df["addresses_full"] = addresses_df["address"].astype(str) + ", " + addresses_df["continent"].astype(str) # combine data from 2 columns
addresses = addresses_df["addresses_full"].values.tolist() # add name of new virtual column to geocode

latitudes = []
longitudes = []
ids = []
g_addresses = []

# geocode each address in file if no ID found at all

if "ids" not in addresses_df: 
	for address in addresses:
		try:
			g = geocoder.geonames(address, key="Mob2023", featureClass='P') # http://www.geonames.org/source-code/javadoc/org/geonames/FeatureClass.html

			if g and len(g):
				geonames_address = g.address
				longitude = g.lng
				latitude = g.lat
				id = g.geonames_id
				#print(geonames_address, longitude, latitude, id) # OPTIONAL: print individual output
			else:
				geonames_address = "NaN"
				longitude = "NaN"
				latitude = "NaN"
				id = "NaN"

			# add information to lists
			g_addresses.append(geonames_address)
			latitudes.append(latitude)
			longitudes.append(longitude)
			ids.append(id)

		except ValueError:
			print("No more data in file.") 
	 
	# write information to new columns in dataframe
	addresses_df["latitudes"] = latitudes
	addresses_df["longitudes"] = longitudes
	addresses_df["ids"] = ids
	addresses_df["geonames address"] = g_addresses
	 
# geocode only empty values if ID column is found

else:
	main_df=addresses_df[addresses_df["ids"].notna()]
	sub_df=addresses_df[addresses_df['ids'].isna()] # fill dataframe where ID cell has no value
	display(sub_df)
	sub_addresses = sub_df["address"].values.tolist()
	for sub_address in sub_addresses:
		try:
			g = geocoder.geonames(sub_address, key="Mob2023", featureClass='P') # http://www.geonames.org/source-code/javadoc/org/geonames/FeatureClass.html

			if g and len(g):
				geonames_address = g.address
				longitude = g.lng
				latitude = g.lat
				id = g.geonames_id
				#print(geonames_address, longitude, latitude, id) # OPTIONAL: print individual output
			else:
				geonames_address = "NaN"
				longitude = "NaN"
				latitude = "NaN"
				id = "NaN"

			# add information to lists
			g_addresses.append(geonames_address)
			latitudes.append(latitude)
			longitudes.append(longitude)
			ids.append(id)

		except ValueError:
			print("No more data in file.")

	# write information to new columns in sub-dataframe
	sub_df["latitudes"] = latitudes
	sub_df["longitudes"] = longitudes
	sub_df["ids"] = ids
	sub_df["geonames address"] = g_addresses

	addresses_df = pd.concat([main_df, sub_df])

print("All addresses geocoded!")


,Unnamed: 0.1,Unnamed: 0,place_old,address,comment_fs,suffix,community,region_1,region_2,continent,variant_1,variant_2,variant_3,Source,addresses_full,latitudes,longitudes,ids,geonames address
3701,3686,3575,Aaminga,Aaminga,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Aaminga, Europe",NaN,NaN,NaN,NaN
3702,3687,3576,Beimberg,Beimberg,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Beimberg, Europe",NaN,NaN,NaN,NaN
3703,3688,3577,Berviana,Berviana,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Berviana, Europe",NaN,NaN,NaN,NaN
3704,3689,3579,Bömbelfürstensis,Bömbelfürst,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Bömbelfürst, Europe",NaN,NaN,NaN,NaN
3705,3690,3580,Borrich,Borrich,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Borrich, Europe",NaN,NaN,NaN,NaN
3707,3692,3584,Bürse,Bürse,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Bürse, Europe",NaN,NaN,NaN,NaN
3708,3695,3588,Collichensis,Collich,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Collich, Europe",NaN,NaN,NaN,NaN
3709,3697,3590,Delbergen,Delbergen,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Delbergen, Europe",NaN,NaN,NaN,NaN
3710,3698,3591,Dürre/Westf,"Dürre, Westfalen",NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Dürre, Westfalen, Europe",NaN,NaN,NaN,NaN
3714,3703,3600,Hederafont,Hederafont,Efeusbrunn? Unklar,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Hederafont, Europe",NaN,NaN,NaN,NaN


All addresses geocoded!


<ipython-input-6-929db1e2caac>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["latitudes"] = latitudes
<ipython-input-6-929db1e2caac>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["longitudes"] = longitudes
<ipython-input-6-929db1e2caac>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

If all addresses have been successfully geocoded, the next step is to check the geocoding and write the results to a new EXCEL file. 


In [ ]:
# view geocoded data
display(addresses_df)

# write geocoded places to new file
addresses_df.to_excel(directory+"Mainz2.xlsx")

addresses_df.to_csv(directory+"Mainz2.csv")

,Unnamed: 0.1,Unnamed: 0,place_old,address,comment_fs,suffix,community,region_1,region_2,continent,variant_1,variant_2,variant_3,Source,addresses_full,latitudes,longitudes,ids,geonames address
0,0,0,Bamberg,Bamberg,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,ProfAPI,"Bamberg, Europe",49.89873,10.90067,2952984.0,Bamberg
1,1,1,?eská Lípa,Böhmisch-Leipa,NaN,NaN,NaN,NaN,NaN,Europe,Böhmisch-Leipa,NaN,NaN,G / 1694 Dezember 7,"Böhmisch-Leipa, Europe",50.68551,14.53764,3077929.0,Česká Lípa
2,2,2,?eský Dub,Böhmisch-Aicha,NaN,NaN,NaN,NaN,NaN,Europe,Böhmisch-Aicha,NaN,NaN,G / 1696 Februar 14,"Böhmisch-Aicha, Europe",50.66054,14.99617,3077895.0,Český Dub
3,3,3,berlingen,Überlingen,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Überlingen, Europe",47.76977,9.17136,2820577.0,Überlingen
4,4,4,A?manstadianus,Assamstadt,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Assamstadt, Europe",49.42806,9.68611,2955050.0,Assamstadt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,3746,3744,Tattelheimensis,Tattelheim,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Tattelheim, Europe",NaN,NaN,NaN,NaN
3749,3747,3747,Tilendropiensis,Boles?awiec,unklar,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Boles?awiec, Europe",NaN,NaN,NaN,NaN
3750,3748,3748,Triltzenbergensis,Triltzenberg,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Triltzenberg, Europe",NaN,NaN,NaN,NaN
3751,3751,3751,Wengeskirchen,Wengeskirchen,fraglich,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,Universitätsmatrikeln,"Wengeskirchen, Europe",NaN,NaN,NaN,NaN


Our geocoded data have been written to a new EXCEL file, which is handy for further (manual) data cleaning and data enrichment. But EXCEL is unfortunately not a file format which GIS applications can handle. This is why we also need to export our geocoded data to GeoJSON.

The conversion of a DataFrame to GeoJSON follows the instructions in the following tutorial by Geoff Boeing:

https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson

In [ ]:
# convert coordinates to floats

addresses_df['latitudes'] = addresses_df['latitudes'].astype(float)
addresses_df['longitudes'] = addresses_df['longitudes'].astype(float)

# ignore places that have not been geocoded

df_geo = addresses_df.dropna(subset=['latitudes', 'longitudes'], axis=0, inplace=False)

# combine information in GeoJSON fromat

def df_to_geojson(df, properties, lat='latitudes', lon='longitudes'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

cols = ['address', 'ids', 'geonames address'] # make sure that your column with place names is selected
geojson = df_to_geojson(df_geo, cols)

with open(directory+'Mainz2.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False)

Your Google Drive should now contain a file with the "geojson" file ending. We can check if this file has been created and if it is well-formed.

In [ ]:
## double-check if GeoJSON file has been created and is well-formed

# load GeoJSON data

with open(directory+'Mainz2.geojson', 'r') as f2:
    data = json.load(f2)
    print(data)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'address': 'Bamberg', 'ids': 2952984.0, 'geonames address': 'Bamberg'}, 'geometry': {'type': 'Point', 'coordinates': [10.90067, 49.89873]}}, {'type': 'Feature', 'properties': {'address': 'Böhmisch-Leipa', 'ids': 3077929.0, 'geonames address': 'Česká Lípa'}, 'geometry': {'type': 'Point', 'coordinates': [14.53764, 50.68551]}}, {'type': 'Feature', 'properties': {'address': 'Böhmisch-Aicha', 'ids': 3077895.0, 'geonames address': 'Český Dub'}, 'geometry': {'type': 'Point', 'coordinates': [14.99617, 50.66054]}}, {'type': 'Feature', 'properties': {'address': 'Überlingen', 'ids': 2820577.0, 'geonames address': 'Überlingen'}, 'geometry': {'type': 'Point', 'coordinates': [9.17136, 47.76977]}}, {'type': 'Feature', 'properties': {'address': 'Assamstadt', 'ids': 2955050.0, 'geonames address': 'Assamstadt'}, 'geometry': {'type': 'Point', 'coordinates': [9.68611, 49.42806]}}, {'type': 'Feature', 'properties': {'address': 'A

Now we can plot the geocoded data to an interactive map. The code below is partly based on an Ipyleaflet Tutorial provided by the *Carpentries Incubator*:

https://carpentries-incubator.github.io/jupyter_maps/01-introduction/index.html

In [ ]:
# plot map from geocoded data and add labels for all places on map

from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML

# Create a map centered on a specific location
map = Map(center=(55, 7), zoom=3)

# print(data) # optional to check if GeoJSON file is read correctly

# Loop over the features and create a marker for each one
for feature in data['features']:
    # Get the coordinates and place name from the feature properties
    coords = feature['geometry']['coordinates']
    place_name = feature['properties']['address'] # double-check name of "place name" column
    #print(place_name) # optional to check if place names are correctly displayed

    # Create a marker

    marker = Marker(location=(coords[1], coords[0]))

    # Create pop-up for each location
        
    popup = Popup(
        location=(coords[1], coords[0]),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False,
    )
    popup.children = str(place_name)[:20] # truncate place names to first "n" characters
    print(popup.children)
    message= HTML()
    message.value = popup.children
    marker.popup = message

    # Add the marker to the map
    map.add_layer(marker)

# Display the map
map



Congratulations, you have just plotted a new map! At the moment, the map only has markers for the point geometries but no pop-up labels. To embed those, other Python packages will need to be imported first. I will add pop-ups in the next development step. 

Notebook created by: Monika Barget

Latest update: 26 January 2023